In [ ]:
# SETUP
from dotenv import load_dotenv
from anthropic import Anthropic

# Load env variables
load_dotenv()

# Create an API client
client = Anthropic()
model="claude-sonnet-4-20250514"

# Make a request

def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)

def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)

def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model":model,
        "max_tokens":1000,
        "messages":messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text